# Test `tweepy` functionality

Steps:
1. Create an app at https://developer.twitter.com/en/apps.

* What is your primary reason for using Twitter developer tools? --> Hobbyist: Exploring the API.
* How will you use the Twitter API or Twitter data? --> I am interested in tracking changes in opinions and attitudes during the coronavirus pandemic to identify events or people who are more influential in driving changes. The code and analysis results will be freely shared on Github.
* Please describe how you will analyze Twitter data including any analysis of Tweets or Twitter users. --> I plan to perform sentiment analysis on tweets from US users after specific events, e.g. new CDC recommendations, shelter-in-place orders, travel restriction announcements. 
* Please describe how and where Tweets and/or data about Twitter content will be displayed outside of Twitter. --> I will share analysis results on my Github page github.com/lucymli/tablecloth and on my blog at lucymli.com.

2. Wait for approval from Twitter.

Twitter requested additional information.

The core use case, intent, or business purpose for your use of the Twitter APIs.
* Use of Twitter APIs would be purely for research purposes.  I am interested in tracking changes in opinions and attitudes during the coronavirus pandemic to identify events or people who are more influential in driving changes.  The code and analysis results will be shared on Github.
If you intend to analyze Tweets, Twitter users, or their content, share details about the analyses you plan to conduct, and the methods or techniques.
* I will produce time series of tweets related to COVID-19 for different locations around the world, train deep learning models to infer sentiment (towards pandemic restrictions), and identify correlations between changes in public health orders/incidence/COVID-19 media reports and changes in sentiment.
If your use involves Tweeting, Retweeting, or liking content, share how you’ll interact with Twitter accounts, or their content.
* I will not tweet, retweet, or like content.
If you’ll display Twitter content off of Twitter, explain how, and where, Tweets and Twitter content will be displayed with your product or service, including whether Tweets and Twitter content will be displayed at row level, or aggregated.
* No individual tweets will be displayed, and no user names will be revealed.  Only aggregated data will be displayed.

3. Go to the API Keys tab, and copy the Consumer Key and Consumer Secret Keys.
> Name your app: twitter_covid_tablecloth

4. Click 'Create my access token' and copy the Access Token and Access Token Secret.
5. Create config file named 'tweepy.ini' that looks like this:

[twitter]\
ConsumerKey = [Consumer Key]\
ConsumerSecret = [Consumer Secret Key]\
AccessToken = [Access Token]\
AccessTokenSecret = [Access Token Secret]

### Set up Twitter authentication

In [15]:
import tweepy
import configparser

config = configparser.ConfigParser()
config.read('tweepy.ini')


auth = tweepy.OAuthHandler(config["twitter"]["ConsumerKey"], config["twitter"]["ConsumerSecret"])
auth.set_access_token(config["twitter"]["AccessToken"], config["twitter"]["AccessTokenSecret"])

api = tweepy.API(auth)

In [11]:
config.read('tweepy.ini').sections()

AttributeError: 'list' object has no attribute 'sections'

### Get public tweets

In [ ]:
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

### Get user information

In [ ]:
user = api.get_user('twitter')

# Using the Coronavirus (COVID-19) Tweets Dataset

In [5]:
from lxml import etree

ModuleNotFoundError: No module named 'lxml'

In [3]:
op = OnePassword()

MissingCredentials: 

In [ ]:
credentials = {'inUserName': 'USERNAME/EMAIL', 'inUserPass': 'PASSWORD'}
url = 'http://www.locationary.com/home/index2.jsp'
requests.post(url, data=payload)

In [36]:
r = requests.get("https://ieee-dataport.org/open-access/coronavirus-covid-19-tweets-dataset#files")

In [37]:
r_text = r.text

In [38]:
r.headers['Content-Type']


'text/html; charset=utf-8'

In [39]:
r_text

'<!doctype html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" version="XHTML+RDFa 1.0" dir="ltr"\n  xmlns:og="http://ogp.me/ns#"\n  xmlns:article="http://ogp.me/ns/article#"\n  xmlns:book="http://ogp.me/ns/book#"\n  xmlns:profile="http://ogp.me/ns/profile#"\n  xmlns:video="http://ogp.me/ns/video#"\n  xmlns:product="http://ogp.me/ns/product#"\n  xmlns:content="http://purl.org/rss/1.0/modules/content/"\n  xmlns:dc="http://purl.org/dc/terms/"\n  xmlns:foaf="http://xmlns.com/foaf/0.1/"\n  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"\n  xmlns:sioc="http://rdfs.org/sioc/ns#"\n  xmlns:sioct="http://rdfs.org/sioc/types#"\n  xmlns:skos="http://www.w3.org/2004/02/skos/core#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n  <head profile="http://www.w3.org/1999/xhtml/vocab">\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n    <!-- Bootstrap CSS -->\n    <link rel="stylesheet" href="https://stackpath.bo